In [ ]:
!pip install pyspark sparknlp numpy scikit-learn tqdm --upgrade transformers torch accelerate

import json
import re
from tqdm import tqdm
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN

# 🔧 Load PhoBERT
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
model = AutoModel.from_pretrained("vinai/phobert-base")
model.eval()

# 📂 Load JSONL data
file_path = "/opt/workspace/data.jsonl"
with open(file_path, "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

# 🎯 Extract user-assistant pairs & clean invalid assistant responses
conversations = []
for item in data:
    messages = item.get("messages", [])
    pair = {}
    for m in messages:
        if m["role"] == "assistant":
            if m.get("content") is None or "can not solve" in m.get("content", "").lower():
                pair = None
                break
            pair["assistant"] = m["content"]
        elif m["role"] == "user":
            pair["user"] = m["content"]
    if pair and "user" in pair and "assistant" in pair:
        conversations.append(pair)

# ✨ Tạo embedding từ user question bằng PhoBERT
def get_embedding(text):
    input_ids = tokenizer.encode(text, return_tensors="pt", max_length=256, truncation=True)
    with torch.no_grad():
        output = model(input_ids)[0]
        embedding = output.mean(dim=1).squeeze().numpy()
    return embedding

questions = [conv["user"] for conv in conversations]
answers = [conv["assistant"] for conv in conversations]

# 🧠 Chuẩn hóa câu hỏi để lọc theo mẫu chung
def normalize_question(text):
    text = text.lower()
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"cho tôi thông tin của [\w\s\.\-]+", "cho tôi thông tin của ...", text)
    text = re.sub(r"có bao nhiêu thiết bị [\w\s]+ tại [\w\s]+", "có bao nhiêu thiết bị ... tại ...", text)
    text = re.sub(r"có bao nhiêu thiết bị [\w\s]+", "có bao nhiêu thiết bị ...", text)
    text = re.sub(r"thiết bị [\w\s]+ có những thuộc tính gì", "thiết bị ... có những thuộc tính gì", text)
    text = re.sub(r"cho tôi biết thông tin", " ", text, flags=re.IGNORECASE)
    text = re.sub(r"cho tôi biết số lượng", " ", text, flags=re.IGNORECASE)
    text = re.sub(r"cho tôi biết thông tin những thiết bị", " ", text, flags=re.IGNORECASE)
    text = re.sub(r"thống kê", " ", text, flags=re.IGNORECASE)
    return text.strip()

normalized_questions = [normalize_question(q) for q in questions]

# 🧹 Lấy chỉ 1 câu đại diện cho mỗi dạng mẫu
unique_indices = {}
for idx, norm_q in enumerate(normalized_questions):
    if norm_q not in unique_indices:
        unique_indices[norm_q] = idx

filtered_questions = [questions[i] for i in unique_indices.values()]
filtered_answers = [answers[i] for i in unique_indices.values()]

# 🧠 Lấy embedding và so sánh độ tương đồng để loại trùng theo nghĩa
embeddings = [get_embedding(q) for q in tqdm(filtered_questions, desc="Embedding")]

similarity_matrix = cosine_similarity(embeddings)
distance_matrix = np.clip(1 - similarity_matrix, 0, None)
dbscan = DBSCAN(metric="precomputed", eps=0.1, min_samples=1).fit(distance_matrix)

# 🧹 Giữ lại 1 câu hỏi đại diện cho mỗi cụm ngữ nghĩa
selected_indices = {label: idx for idx, label in enumerate(dbscan.labels_)}.values()
cleaned_data = []
for i in selected_indices:
    cleaned_data.append({
        "messages": [
            {"role": "user", "content": filtered_questions[i]},
            {"role": "assistant", "content": filtered_answers[i]}
        ]
    })

# 💾 Xuất ra file
output_path = "/opt/workspace/clean_conversations.jsonl"
with open(output_path, "w", encoding="utf-8") as f:
    for item in cleaned_data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print(f"✅ Đã lưu file sạch tại: {output_path}")
